In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy
import geopandas as gpd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib

pd.options.display.max_rows = 500
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')

# import categories as cats
# import constraints

In [531]:
from extended_survey import process_people_df, process_places_df, nivacad_posmap
from census import process_census
from constraints import get_ind_const, get_viv_const

from setup_lin_system import setup_ls, check_solvable, find_conf_const, get_X, get_X_I, get_W, find_zero_nozero_const, find_nonzero_zero_const
from setup_lin_system import get_X_I_fast, get_matrices, fill_zero_h, make_init_dict, fix_zero_cell, make_init_system, fix_zero_cell_all
from census import locate_collective

In [3]:
survey_dir = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/')
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')
output_path = Path('../output/')

In [4]:
personas_census_controlled = [
    'SEXO', 'EDAD', 'ENT_PAIS_NAC', 'AFRODES', 'DHSERSAL', 'RELIGION', 'DIS',
    'HLENGUA', 'ASISTEN', 'NIVACAD', 'ALFABET', 'ENT_PAIS_RES_5A', 'SITUA_CONYUGAL',
    'CONACT',
]

personas_to_drop = [
    # Misc
    'NACIONALIDAD', 'SERSALUD', 'ELENGUA', 'PERTE_INDIGENA',     
]

personas_extra = [
    # Household related
    'CLAVIVP', 'PARENTESCO',
    
    # Work related   
    'INGTRMEN', 'OCUPACION_C', 'SITTRA', 'HORTRA', 'ACTIVIDADES_C',
    'AGUINALDO', 'VACACIONES', 'SERVICIO_MEDICO', 'UTILIDADES', 'INCAP_SUELDO', 'SAR_AFORE', 'CREDITO_VIVIENDA',

    # Transport related
    'ENT_PAIS_ASI',
    'MED_TRASLADO_ESC_Automóvil o camioneta',
    'MED_TRASLADO_ESC_Bicicleta',
    'MED_TRASLADO_ESC_Blanco por pase',
    'MED_TRASLADO_ESC_Caminando',
    'MED_TRASLADO_ESC_Camión, autobús, combi, colectivo',
    'MED_TRASLADO_ESC_Metro, tren ligero, tren suburbano',
    'MED_TRASLADO_ESC_Metrobús (autobús en carril confinado)',
    'MED_TRASLADO_ESC_Motocicleta o motoneta',
    'MED_TRASLADO_ESC_No especificado',
    'MED_TRASLADO_ESC_Otro',
    'MED_TRASLADO_ESC_Taxi (App Internet)',
    'MED_TRASLADO_ESC_Taxi (sitio, calle, otro)',
    'MED_TRASLADO_ESC_Transporte escolar',
    
    'ENT_PAIS_TRAB',
    'MED_TRASLADO_TRAB_Automóvil o camioneta',
    'MED_TRASLADO_TRAB_Bicicleta',
    'MED_TRASLADO_TRAB_Blanco por pase',
    'MED_TRASLADO_TRAB_Caminando',
    'MED_TRASLADO_TRAB_Camión, autobús, combi, colectivo',
    'MED_TRASLADO_TRAB_Metro, tren ligero, tren suburbano',
    'MED_TRASLADO_TRAB_Metrobús (autobús en carril confinado)',
    'MED_TRASLADO_TRAB_Motocicleta o motoneta',
    'MED_TRASLADO_TRAB_No especificado',
    'MED_TRASLADO_TRAB_Otro',
    'MED_TRASLADO_TRAB_Taxi (App Internet)',
    'MED_TRASLADO_TRAB_Taxi (sitio, calle, otro)',
    'MED_TRASLADO_TRAB_Transporte escolar',
]

# This should change when cloning across municipalities
spatial_cols_per = [
    'MUN_ASI',
    'TIE_TRASLADO_ESCU',

    'MUN_TRAB',
    'TIE_TRASLADO_TRAB',
    
    'MUN_RES_5A',
    'MUN',
]

viviendas_census_controlled = [
    'PISOS', 'ELECTRICIDAD', 'AGUA_ENTUBADA', 'ABA_AGUA_ENTU',
    'TINACO', 'CISTERNA', 'SERSAN', 'CONAGUA', 'DRENAJE',
    
    'REFRIGERADOR', 'LAVADORA', 'HORNO',
    'RADIO', 'TELEVISOR', 'COMPUTADORA', 'TELEFONO', 'CELULAR',
    'INTERNET', 'SERV_TV_PAGA', 'SERV_PEL_PAGA', 'CON_VJUEGOS',
    
    'AUTOPROP', 'MOTOCICLETA', 'BICICLETA',
    
    'CUADORM', 'TOTCUART',
    
    'JEFE_SEXO'
]

viviendas_extra = [
    'CLAVIVP', 'NUMPERS', 'TIPOHOG', 'JEFE_EDAD',
    'INGTRHOG'
]

viviendas_to_drop = [
    'PAREDES', 'TECHOS', 'COCINA', 'LUG_COC', 'COMBUSTIBLE',
    'ESTUFA', 'FOCOS', 'FOCOS_AHORRA', 'ABA_AGUA_NO_ENTU', 'BOMBA_AGUA',
    'REGADERA', 'BOILER', 'CALENTADOR_SOLAR', 'AIRE_ACON', 'PANEL_SOLAR',
    'USOEXC', 'SEPARACION1', 'SEPARACION2', 'SEPARACION3', 'SEPARACION4', 
    'DESTINO_BAS', 'TENENCIA', 'ESCRITURAS', 'FORMA_ADQUI', 
    
    'FINANCIAMIENTO1', 'FINANCIAMIENTO2', 'FINANCIAMIENTO3', 'DEUDA',
    
    'MCONMIG', 'MNUMPERS',
    
    'INGR_PEROTROPAIS', 'INGR_PERDENTPAIS', 'INGR_AYUGOB', 'INGR_JUBPEN',
    
    'ALIMENTACION', 'ALIM_ADL1', 'ALIM_ADL2',
    'ING_ALIM_ADL1', 'ING_ALIM_ADL2', 'ING_ALIM_ADL3'
]

# This should change when cloning across municipalities
spatial_cols_viv = [
    'MUN'
]

In [5]:
%%time
constraints_ind = get_ind_const()
constraints_viv = get_viv_const()

viviendas_cat = process_places_df(viviendas_path, to_drop=viviendas_to_drop)
viviendas_full = pd.concat(viviendas_cat.values())

personas_cat = process_people_df(personas_path)
personas_full = pd.concat(personas_cat.values())

(
    df_mun, df_mun_min, df_mun_max,
    df_loc, df_loc_min, df_loc_max,
    df_agebs, df_agebs_min, df_agebs_max
) = process_census(census_iter_path, census_resageburb_path)

met_zone = [
    "Monterrey",
    "Guadalupe",
    "Apodaca",
    "San Nicolás de los Garza",
    "General Escobedo",
    "Santa Catarina",
    "Juárez",
    "García",
    "Pesquería",
    "San Pedro Garza García",
    "Cadereyta Jiménez",
    "Santiago",
    "Salinas Victoria",
    "Hidalgo",
    "General Zuazua",
    "Ciénega de Flores",
    "El Carmen",
    "Abasolo",
]

CPU times: user 20.6 s, sys: 1.74 s, total: 22.3 s
Wall time: 22.3 s


In [65]:
pea_tot = df_mun.loc[met_zone].PEA.sum()

pea = personas_full[(personas_full.MUN.isin(met_zone)) & (personas_full.CONACT=='Trabaja')]

pea_counts = pea.groupby('ACTIVIDADES_C', observed=True).FACTOR.sum()

pea_est = pea_counts.sum()

pea_adj = round(pea_counts * pea_tot / pea_est)

pea_adj.to_csv('actividads_c_censo.csv')

In [659]:
%%time
viviendas, X, I, U, C, Y = make_init_system(personas_full, viviendas_full, constraints_ind, constraints_viv, df_mun)

CPU times: user 33.3 s, sys: 618 ms, total: 33.9 s
Wall time: 33.9 s


In [319]:
%%time
check_solvable(U.values, C.sum().values)

CPU times: user 19.8 s, sys: 363 ms, total: 20.2 s
Wall time: 3.37 s


True

In [660]:
%%time
Y_fix = fix_zero_cell_all(Y, U, C)

CPU times: user 847 ms, sys: 153 ms, total: 1e+03 ms
Wall time: 1.04 s


In [646]:
# Get a dictionary with all possible columns categories.
cat_personas = {}
for col in X.columns:
    if col == 'FACTOR':
        continue
    cat_personas[col] = X[col].cat.categories.tolist()
cat_viviendas = {}
for col in viviendas_full.columns:
    if viviendas_full[col].dtype != 'category':
        continue
    cat_viviendas[col] = viviendas_full[col].cat.categories.tolist()

In [893]:
mun = 'Abasolo'

mask = Y_fix.loc[:, mun] > 0
Ur = U.loc[:, mask]

conf_consts = find_conf_const(Ur, C.loc[mun])
conf_consts

[['VPH_2YMASD', 'VPH_PISODT', 'VPH_PISOTI', 'VPH_1DOR'],
 ['VPH_NEDOR', 'TOTHOG', 'VPH_PISODT', 'VPH_PISOTI']]

In [831]:
Y_ext = Y_fix.copy()

In [866]:
Y_ext = Y_fix.copy()

# Find all combinations of conflicting constraints sets
combs = set()
for consts in conf_consts:
    # Constraints are people or household?
    are_people = consts[0] in cat_personas
    if are_people:
        const_dict = constraints_ind
        cat_dict = cat_personas
    else:
        const_dict = constraints_viv
        cat_dict = cat_viviendas

    # Find columns involved in the constraints
    cols = []
    for const in consts:
        cols.extend(list(const_dict[const].keys()))
    cols = list(set(cols))

    # Find all combinations of cols, ignore Blanco por pase
    ccombs = product(*[cat_dict[col] for col in cols])
    ccombs = set([c for c in ccombs if 'Blanco por pase' not in c])
    combs.update(ccombs)

consts = list(set(sum(conf_consts, [])))
# Find all combinations already present in mun
mask_in = Y_ext[mun] > 0
viviendas_in = viviendas.loc[mask_in]
combs_in = set(zip(*[viviendas_in[c] for c in cols]))

# Find missing combinations in mun
combs_miss = combs - combs_in

# Build query to search for them
query_l = []
for comb in combs_miss:
    query_l.append(f'({cols[0]}=="{comb[0]}" & {cols[1]}=="{comb[1]}")')
query = ' | '.join(query_l)

# Get viv ids
mask_miss = viviendas.eval(query)
assert mask_miss.sum() > 0
    
# Solve restrictred nnls problem
mask = mask_in | mask_miss
Cr = C.loc[mun, consts]
Ur = U.loc[consts, mask]
Yr, err = scipy.optimize.nnls(Ur, Cr)
assert err < 1e-10
mask0 = np.zeros_like(mask, dtype=bool)
mask0[mask] = Yr.astype(bool)

# Find nnls solutions belonging to missing set
mask_ext = mask0 & mask_miss
n_ext = mask_ext.sum()
assert n_ext > 0

# Add them to seed with unitary weight
Y_ext.loc[mask_ext, mun] = 1

In [920]:
# Find all combinations of conflicting constraints sets
combs = []
for consts in conf_consts:
    # Constraints are people or household?
    are_people = consts[0] in cat_personas
    if are_people:
        const_dict = constraints_ind
        cat_dict = cat_personas
    else:
        const_dict = constraints_viv
        cat_dict = cat_viviendas

    # Find columns involved in the constraints
    cols = []
    for const in consts:
        cols.extend(list(const_dict[const].keys()))
    cols = tuple(set(cols))

    # Find all combinations of cols, ignore Blanco por pase
    ccombs = product(*[cat_dict[col] for col in cols])
    ccombs = set([c for c in ccombs if 'Blanco por pase' not in c])
    combs.extend([(cols, ccomb) for ccomb in ccombs])
combs = set(combs)
combs_dict = defaultdict(list)
for cols, comb in combs:
    combs_dict[cols].append(comb)

# Get full list of constraints
consts = list(set(sum(conf_consts, [])))

# Find all combinations already present in mun
# as well as missing combinations
mask_in = Y_ext[mun] > 0
viviendas_in = viviendas.loc[mask_in]
combs_in_dict = defaultdict(list)
combs_miss_dict = defaultdict(list)
for cols, combs in combs_dict.items():
    combs_in_dict[cols] = set(zip(*[viviendas_in[c] for c in cols]))
    combs_miss_dict[cols] = set(combs_dict[cols]) - combs_in_dict[cols]

# Build query to search for them
query_l = []
for cols, combs in combs_miss_dict.items():
    query_l.append(f'({cols[0]}=="{comb[0]}" & {cols[1]}=="{comb[1]}")')
query = ' | '.join(query_l)

In [921]:
combs_miss_dict

defaultdict(list,
            {('PISOS', 'CUADORM'): {('No Tierra', 'No especificado'),
              ('No especificado', '2+'),
              ('No especificado', 'No especificado'),
              ('No especificado', 1),
              ('Tierra', 'No especificado')}})

In [905]:
combs_in_dict

defaultdict(list,
            {('PISOS', 'CUADORM'): {('Blanco por pase', 'Blanco por pase'),
              ('No Tierra', '2+'),
              ('No Tierra', 1),
              ('Tierra', '2+'),
              ('Tierra', 1)}})

In [903]:
for cols, combs in combs_dict.items():
    print(combs)

[('No especificado', 'No especificado'), ('No especificado', 1), ('No Tierra', 'No especificado'), ('No Tierra', 1), ('Tierra', '2+'), ('Tierra', 'No especificado'), ('No Tierra', '2+'), ('No especificado', '2+'), ('Tierra', 1)]


In [867]:
(Y_ext[mun] - Y_fix[mun]).sum()

1.0

In [868]:
mun = 'Abasolo'

mask = Y_ext.loc[:, mun] > 0
Ur = U.loc[:, mask]

conf_consts = find_conf_const(Ur, C.loc[mun])
conf_consts

[['VPH_3YMASC', 'VPH_1DOR', 'VPH_2YMASD', 'VPH_1CUART', 'VPH_2CUART'],
 ['VPH_NECUART', 'TOTHOG', 'VPH_1DOR', 'VPH_2YMASD'],
 ['VPH_S_ELEC', 'VPH_1DOR', 'VPH_2YMASD', 'VPH_C_ELEC'],
 ['VPH_NE_ELEC', 'TOTHOG', 'VPH_1DOR', 'VPH_2YMASD'],
 ['VPH_AGUAFV', 'VPH_1DOR', 'VPH_2YMASD', 'VPH_AGUADV'],
 ['VPH_AGUANE', 'TOTHOG', 'VPH_1DOR', 'VPH_2YMASD'],
 ['VPH_NODREN', 'VPH_1DOR', 'VPH_2YMASD', 'VPH_DRENAJ'],
 ['VPH_NEDREN', 'TOTHOG', 'VPH_1DOR', 'VPH_2YMASD']]

In [688]:
# This is not a restricterd nnls system, we need to consider the already present vivs toguether with new candidates
# nnls may assign zero weight to an origianal viv, besides the number of original vivs may be quite large

VPH_2YMASD    522.0
VPH_PISODT    792.0
VPH_PISOTI      6.0
VPH_1DOR      277.0
Name: Abasolo, dtype: float64

In [ ]:
# Now, what about conflicting constraints?
# We can use the nnls solution to add households sparsely
# But the weights?

# Sketch of the solution...
# 1. Find conflicting constraints Done
# 2. Identofy households and people datframes among all municipalities as in the zero constraints solution
# 3. Add an identifuer column, extra = 1
# 4. Merge with households involved in the constraints already in mun_dest
# 5. Find the nnnls solution.
# 6. Keep the non-zero rows of new households.
# 7. Add each a weight of 1 (extend sample). Make sure there are few additions.
# 8. Assert the conflicting constraints are no more. If this is not the case, the nnls solution is no good.

In [ ]:
def f(consts):
    assert len(const) > 0
    
    # 
    
    
    # Get columns involved in the constraints
    
    # Get all possible combinations of the involved columns
    
    # Get all combinations already present in mun
    
    # Remove them from combination list
    
    # Look for households with needed combinations
    
    # Merge households with existing combs with households with new combs
    
    # Solve the restricted nnls problem
    
    # Keep only new households with positive weights
    
    # Return extended raw list of households and people


In [107]:
query_l = []
for comb in combs:
    if comb in combs_orig:
        continue
    query_l.append(f'({cols[0]}=="{comb[0]}" & {cols[1]}=="{comb[1]}")')
query = ' | '.join(query_l)
query

'(PISOS=="No Tierra" & CUADORM=="No especificado") | (PISOS=="Tierra" & CUADORM=="No especificado") | (PISOS=="No especificado" & CUADORM=="1") | (PISOS=="No especificado" & CUADORM=="2+") | (PISOS=="No especificado" & CUADORM=="No especificado")'

In [108]:
viviendas_full.query(query)

,FACTOR,MUN,CLAVIVP,PISOS,CUADORM,TOTCUART,ELECTRICIDAD,AGUA_ENTUBADA,ABA_AGUA_ENTU,TINACO,...,CELULAR,INTERNET,SERV_TV_PAGA,SERV_PEL_PAGA,CON_VJUEGOS,NUMPERS,TIPOHOG,INGTRHOG,JEFE_SEXO,JEFE_EDAD
ID_VIV,,,,,,,,,,,,,,,,,,,,,
190040000210,9,Allende,Casa que comparte terreno con otra(s),No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,Sí,Sí,Sí,4,Hogar Nuclear (Familiar),"20,000-39,999",M,25-49
190040000267,7,Allende,Casa única en el terreno,No Tierra,No especificado,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,Sí,...,Sí,Sí,No,Sí,No,3,Hogar Nuclear (Familiar),"10,000-19,999",M,25-49
190060001761,37,Apodaca,Casa única en el terreno,No Tierra,No especificado,No especificado,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,Sí,Sí,No,5,Hogar Nuclear (Familiar),No especificado,M,25-49
190060002626,37,Apodaca,Casa única en el terreno,No Tierra,No especificado,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,No,No,No,No,2,Hogar Ampliado (Familiar),No especificado,M,25-49
190100001195,4,El Carmen,Casa única en el terreno,No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,Sí,Sí,Sí,3,Hogar Nuclear (Familiar),No especificado,M,50-59
190100001379,5,El Carmen,Casa única en el terreno,No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,Sí,...,Sí,Sí,No,No,Sí,4,Hogar Nuclear (Familiar),No especificado,M,25-49
190100001390,5,El Carmen,Casa única en el terreno,No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,No,No,No,3,Hogar Ampliado (Familiar),"10,000-19,999",M,25-49
190100001498,5,El Carmen,Casa única en el terreno,No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,Sí,No,No,4,Hogar Nuclear (Familiar),"5,000-9,999",M,25-49
190100001740,5,El Carmen,Casa única en el terreno,No especificado,2+,3+,Sí,Tienen agua entubada.,Del servicio público de agua.,No,...,Sí,Sí,No,Sí,Sí,4,Hogar Nuclear (Familiar),"20,000-39,999",F,25-49


In [20]:
for mun in personas_cat.keys():
    if len(XWC_ext[mun]['conf_consts'])==0: continue
    print(mun, XWC_ext[mun]['conf_consts'])
    print()

Abasolo [['VPH_2YMASD', 'VPH_PISODT', 'VPH_PISOTI', 'VPH_1DOR'], ['VPH_NEDOR', 'TOTHOG', 'VPH_PISODT', 'VPH_PISOTI']]

Los Aldamas [['PSIN_RELIG', 'POBTOT', 'P_12YMAS', 'P_12YMAS_M', 'PEA_F', 'PE_INAC_F', 'PCATOLICA', 'PRO_CRIEVA'], ['PRELIG_NE', 'P_12YMAS', 'P_12YMAS_M', 'PEA_F', 'PE_INAC_F']]

Allende [['VPH_S_ELEC', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC', 'VPH_C_ELEC'], ['VPH_NE_ELEC', 'TOTHOG', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC'], ['VPH_AGUAFV', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC', 'VPH_AGUADV'], ['VPH_AGUANE', 'TOTHOG', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC'], ['VPH_NODREN', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC', 'VPH_DRENAJ'], ['VPH_NEDREN', 'TOTHOG', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]

Anáhuac [['VPH_2YMASD', 'VPH_PISODT', 'VPH_PISOTI', 'VPH_1DOR'], ['VPH_NEDOR', 'TOTHOG', 'VPH_PISODT', 'VPH_PISOTI'], ['VPH_3YMASC', 'VPH_PISODT', 'VPH_PISOTI', 'VPH_1CUART', 'VPH_2CUART'], ['VPH_NECUART', 'TOTHOG', 'VPH_PISODT', 'VPH_PISOTI'], ['VPH_S_ELEC', 'VPH_PISODT'

## Collective

In [11]:
df_mun_loc = locate_collective(df_mun, df_loc)

In [12]:
for mun, n_p, n_v, df in df_mun_loc:
    print(mun, n_p, n_v, df.index.values)

7 8.0 1.0 [ 29 216 219 221 269 344]
9 1873.0 1.0 [620]
31 10.0 1.0 [160 298]
49 4.0 1.0 [ 22  57  58  67  77 127 128 138 139 149 180 196 197 206 209 233 236 254
 272 278 289 295 296]


## Different type of solutions

In [16]:
import scipy.linalg as spl
from scipy.optimize import nnls

In [11]:
mun = 'Cerralvo'
X2 = XWC_dict[mun]['X']
W2 = XWC_dict[mun]['W']
C2 = XWC_dict[mun]['C']

check_solvable(W2, C2)

True

In [106]:
W.shape, W2.shape

((97, 2882), (183, 2882))

In [90]:
# Non negative least squares solution

X_sol_nn, err = nnls(W, C)
np.linalg.norm(W.values @ X_sol_nn - C.values, ord=1), np.linalg.norm(X_sol_nn)

(6.67119692820961e-11, 1280.5252064123974)

In [91]:
X_sol_nn2, err = nnls(W2, C2)
np.linalg.norm(W2.values @ X_sol_nn2 - C2.values, ord=1), np.linalg.norm(X_sol_nn2)

(9.615463980594541e-11, 1091.7601945594413)

In [100]:
# Least squares solution

W_inv = spl.pinv(W)
X_sol = W_inv @ C.values
np.linalg.norm(W.values @ X_sol - C.values, ord=1), np.linalg.norm(X_sol)

(2.957613265408554e-09, 174.80568107488986)

In [104]:
# Least squares solution

W_inv = spl.pinv(W2)
X_sol2 = W_inv @ C2.values
np.linalg.norm(W2.values @ X_sol2 - C2.values, ord=1), np.linalg.norm(X_sol2)

(1.2072849386868256e-09, 174.805681074888)

In [105]:
np.linalg.norm(X_sol2 - X_sol)

5.845229258204539e-10

In [518]:
from ipf import ipf_classic_numba

In [107]:
%%time
# The IPF solution
x = X.FACTOR.values.astype(float)
x_ipf, err = ipf_classic_numba(x, W.values.astype(float), C.values, tol=1e-1, max_iters=1e4)
err, np.linalg.norm(x_ipf)

CPU times: user 7.97 s, sys: 791 ms, total: 8.76 s
Wall time: 6.36 s


(44.4800477607044, 205.72287053844428)

In [115]:
%%time
# The IPF solution
x = X2.FACTOR.values.astype(float)
x_ipf2, err = ipf_classic_numba(x, W2.values.astype(float), C2.values, tol=1e-3, max_iters=1e4)
err, np.linalg.norm(x_ipf2)

CPU times: user 1.21 s, sys: 200 ms, total: 1.41 s
Wall time: 1.15 s


(0.0009983262271617832, 206.0888458338842)

In [681]:
# The max ent solution

### OD Survey

In [96]:
od_df = pd.read_csv('../data/OD_Survey/datos_limpios_tiempos.csv', low_memory=False)

In [101]:
od_df.head()

,ID-HOGAR,H-P,H-P-V,Latitud,Longitud,FechaHoraEnc,NumVisita,TipoEnc,RealizoEnc,Encuestador,Supervisor,Cod_MunDomicilio,ColDom,CalleDom,NExtDom,NIntDom,RefDom,CPDom,Punto_zona,Cod_EdoDomicilio,LineaTelef,VHAuto,VHMoto,VHPickup,VHCamion,VHBici,VHPatineta,VHPatines,VHScooter,VHOtro,CHBaños,CHDormitorios,Internet,Hab14masTrabajo,HabitantesTotal,HbitantesMayor6,HbitantesMenor5,TodosEstan,NunHabitante,Género,Edad,RelaciónHogar,RelaciónHogar_O,Discapacidad,Discapacidad_O,Estudios,Estudios_O,Ocupacion,Ocupacion_O,SectorEconom,SectorEconom_O,ViajeAyer,Num_Viaje,Lugar_Or,Cod_MunOri,ColOri,RefOri,CalleOri,Esquina_Ori,Cruce_Ori,OtroEstadoOri,OtroEstadoOri_O,CodOri,Cod_EdoOri,Macrozona Origen,ZonaOri,Hora Inicio V,LugarDest,Cod_MunDest,ColDest,RefDest,CalleDest,Esquina_Dest,Cruce_Dest,OtroEstadoDest,OtroEstadoDest_O,CodDest,Cod_EdoDest,Macrozona Destino,ZonaDest,Hora Término Viaje,Cod_IDEdoDest,Cod_IDMunDest,Cod_IDLocDest,Cod_LocDest,Cod_IDColDest,Cod_ColDest,Cod_IDRefDest,Cod_RefDest,Tiempo Tot de Viaje,Motivo,Motivo_O,M1_TipoTransp,M1_Transp_O,M1Tpo_Caminata,M1N_Ruta,M1_HHTpoParada,M1_MMTpoParada,M1_HHTpoAbordo,M1_HHTpoAbordo_O,M1_MMTpoAbordo,M1_Pago,M2_Transp,M2_Transp_O,M2_TipoTransp,M2Tpo_Caminata,M2N_Ruta,M2_HHTpoParada,M2_MMTpoParada,M2_TpoTranspordo,M2_HHTpoAbordo,M2_HHTpoAbordo_O,M2_MMTpoAbordo,M2_Pago,M3_Transp,M3_Transp_O,M3_TipoTransp,M3Tpo_Caminata,M3N_Ruta,M3_HHTpoParada,M3_MMTpoParada,M3_TpoTranspordo,M3_HHTpoAbordo,M3_HHTpoAbordo_O,M3_MMTpoAbordo,M3_Pago,M4_Transp,M4_Transp_O,M4_TipoTransp,M4Tpo_Caminata,M4N_Ruta,M4_HHTpoParada,M4_MMTpoParada,M4_TpoTranspordo,M4_HHTpoAbordo,M4_HHTpoAbordo_O,M4_MMTpoAbordo,M4_Pago,M5_Transp,M5_Transp_O,M5_TipoTransp,M5Tpo_Caminata,M5N_Ruta,M5_HHTpoParada,M5_MMTpoParada,M5_TpoTranspordo,M5_HHTpoAbordo,M5_HHTpoAbordo_O,M5_MMTpoAbordo,M5_Pago,M6_Transp,M6_Transp_O,M6_TipoTransp,M6Tpo_Caminata,M6N_Ruta,M6_HHTpoParada,M6_MMTpoParada,M6_TpoTranspordo,M6_HHTpoAbordo,M6_HHTpoAbordo_O,M6_MMTpoAbordo,M6_Pago,M7_Transp,M7_Transp_O,M7_TipoTransp,M7Tpo_Caminata,M7N_Ruta,M7_HHTpoParada,M7_MMTpoParada,M7_TpoTranspordo,M7_HHTpoAbordo,M7_HHTpoAbordo_O,M7_MMTpoAbordo,M7_Pago,M8_Transp,M8_Transp_O,M8_TipoTransp,M8Tpo_Caminata,M8N_Ruta,M8_HHTpoParada,M8_MMTpoParada,M8_TpoTranspordo,M8_HHTpoAbordo,M8_HHTpoAbordo_O,M8_MMTpoAbordo,M8_Pago,TipoEstacionamiento,TpoBusqueda,TpoEstacionadoHH,TpoEstacionadoMM,CostoEstacionamiento,Obs_Encuestador,Obs_ENCUESTA,FE,Modo Agrupado,motivos,genero,estudios,disc,origen,Tiempo,tiempo_s,tiempo_m,tiempo_h
0,23853a-20,23853a-20/2,23853a-20/2-2,25.765088,-100.406374,2019-09-24T00:00:00Z,Visita 1,HABITANTE,NaN,Lucia Hernández Mondragón,Patricia Baena,Monterrey,Barrio Estrella Elit,lucero,112,NaN,NaN,64102.0,86,Nuevo Leon,Si,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Si,4.0,4.0,4.0,NaN,NaN,2,Mujer,40,Madre/Esposa,NaN,Ninguna,NaN,Sin Instrucción,NaN,Ama de casa,NaN,Otro,NaN,Sí,2,El DESTINO de viaje inmediato anterior,Monterrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10358.0,Nuevo Leon,19.0,76,1899-12-31T18:50:00Z,El HOGAR,Monterrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2249.0,Nuevo Leon,17.0,86,1899-12-31T19:00:00Z,19.0,39.0,NaN,NaN,34979,BARRIO ESTRELLA,NaN,NaN,1899-12-31T00:10:00Z,regreso a casa,NaN,Vehículo Particular,NaN,0.0,NaN,0.0,0,0,0.0,10,NaN,No utilizó otro medio de transporte,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"Propio (Casa, oficina)",NaN,11.0,NaN,NaN,NaN,NaN,1282.883061,automovil,3,F,Bajo,No,NA+NA,NaN,600,10,0.166667
1,23853a-20,23853a-20/4,23853a-20/4-2,25.765088,-100.406374,2019-09-24T00:00:00Z,Visita 1,VIAJE,NaN,Lucia Hernández Mondragón,Patricia Baena,Monterrey,Barrio Estrella Elit,lucero,112,NaN,NaN,64102.0,86,Nuevo Leon,Si,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Si,4.0,4.0,4.0,NaN,NaN,4,Hombre,

### Determining household structure

In [17]:
personas.columns

Index(['ID_PERSONA', 'ID_VIV', 'FACTOR', 'MUN', 'CLAVIVP', 'SEXO', 'EDAD',
       'ENT_PAIS_NAC', 'PARENTESCO', 'NACIONALIDAD', 'SERSALUD', 'AFRODES',
       'DHSERSAL', 'RELIGION', 'DIS', 'HLENGUA', 'ELENGUA', 'PERTE_INDIGENA',
       'ASISTEN', 'ENT_PAIS_ASI', 'MUN_ASI', 'TIE_TRASLADO_ESCU',
       'MED_TRASLADO_ESC_Automóvil o camioneta', 'MED_TRASLADO_ESC_Bicicleta',
       'MED_TRASLADO_ESC_Blanco por pase', 'MED_TRASLADO_ESC_Caminando',
       'MED_TRASLADO_ESC_Camión, autobús, combi, colectivo',
       'MED_TRASLADO_ESC_Metro, tren ligero, tren suburbano',
       'MED_TRASLADO_ESC_Metrobús (autobús en carril confinado)',
       'MED_TRASLADO_ESC_Motocicleta o motoneta',
       'MED_TRASLADO_ESC_No especificado', 'MED_TRASLADO_ESC_Otro',
       'MED_TRASLADO_ESC_Taxi (App Internet)',
       'MED_TRASLADO_ESC_Taxi (sitio, calle, otro)',
       'MED_TRASLADO_ESC_Transporte escolar', 'NIVACAD', 'ALFABET',
       'ENT_PAIS_RES_5A', 'MUN_RES_5A', 'SITUA_CONYUGAL', 'CONACT',
       'OC

In [27]:
# Map people to households
viv_per_dict = {}
for id_viv, group in personas_full.groupby('ID_VIV'):
    viv_per_dict[id_viv] = group[['ID_PERSONA', 'SEXO', 'EDAD', 'PARENTESCO']]

In [52]:
household_set = set()
for viv_id, viv_df in viv_per_dict.items():
    # print(tuple(viv_df.PARENTESCO.value_counts()))
    household_set.update([tuple(viv_df.PARENTESCO.value_counts())])

In [53]:
len(household_set)

189

In [61]:
viv_per_dict[190010000001]

,ID_PERSONA,SEXO,EDAD,PARENTESCO
0,19001000000100001,F,8-11,Hija(o)
1,19001000000100002,M,25-49,Esposa(o)
2,19001000000100003,M,0-2,Hija(o)
3,19001000000100004,F,6-7,Hija(o)
4,19001000000100005,F,25-49,Jefa(e)


In [ ]:
# number of workers
# child by age

In [62]:
personas.PARENTESCO.value_counts()

Hija(o)                                        1149
Jefa(e)                                         791
Esposa(o)                                       535
Nieta(o)                                        252
Nuera o yerno                                    69
Sobrina(o)                                       36
Hermana(o)                                       28
Madre o padre                                    18
Hijastra(o)                                      14
Cuñada(o)                                        12
Bisnieta(o) o tataranieta(o)                      7
Otros familiares                                  7
Suegra(o)                                         6
Parentesco no especificado                        4
Trabajador(a) doméstico(a)                        2
Sin parentesco                                    2
Hija(o) adoptiva(o)                               1
Esposa(o) de nieto(a)                             1
Concubina(o) o unión libre                        1
Madrastra o 

### Seed from group counts

In [ ]:
# Seed for whole met area

In [ ]:
# Seed per municipality

In [ ]:
# Comparing seeds from municipality and whole area

### Seed using bayesian networks

### Comparing both seeds

# Reconstrucción de microdatos para el censo 2020, nivel AGEB

El objetivo es obtener una base de microdatos para cada AGEB del censo 2020 consistente con los tabulados oficiales.
Para tal efecto, vamos a probar diferentes metodologías, comenzando con una estrategia de reconstrucción utilizando métodos de programación lineal.

Para tener una idea clara de la implementación, tomemos una AGEB específica de la zona metropolitana de Monterrey.

A nivel municipio o demarcación territorial, localidad y AGEB, cualquier indicador con menos de tres unidades aparece con asterisco
a excepción de las variables Población total (POBTOT), Total de viviendas (VIVTOT) y Total de viviendas habitadas (TVIVHAB).

Para comenzar vamos a restringir nuestro objetivos a dos variables: edad y sexo. Usemos una AGEB sin datos faltantes, más adelante ajustaremos la implementación para contemplar los datos redactados (*).

Porbaremos con dos AGEBs,  una será la de menor población con datos columnos para las columnas que se refieren a población y sexo, la segunda la de mayor población.

Las columnas de población en la tabla de microdatos serán discretizadas en el menor número intervalos compatibles con las categorías del censo que puedan aprovechar las restricciones.
La ventaja de no usar una única columna de edad con valor entero es la disminución de tamaño del espacio de soluciones.
Imputar una edad específica puede hacerse en una etapa de procesamiento posterior.
Para no contar dos veces soluciones que unicamente difieren en el ordenamiento de las filas, agregaremos restricciones de ordenamiento para los valores de edad.

Las categorias posibles de obtener del censo son las siguientes y las codificamos como enteros:
- 0-2: 0
- 3-4: 1
- 5: 2
- 6-7: 3
- 8-11: 4
- 12-14: 5
- 15-17: 6
- 18-24: 7
- 25-59: 8
- 60-64: 9
- 65-130: 10

Tener esta codificación en mente es importante al momento de definir las restricciones.

Al momento de crear los objetivos, usaremos un arreglo de variables en 2D, la tabla de microdatos.
Esto debe ser compatible con OR-Tools, por lo que será implementado como una lista de listas.
Es este problema restringido a edad y sexo, necesitamos dos columnas: SEXO y EDAD.

In [40]:
def series_to_dict(series):
    d = series.to_dict()
    keys = d.keys()
    
    vals = d.values()
    vals = [int(v) 
            if isinstance(v, float) and not np.isnan(v)
            else v for v in d.values() ]
    
    d = {k: v for k,v in zip(keys, vals)}
    
    return d

## Codificando directamente la tabla de contingencia

In [43]:
from ortools.sat.python import cp_model
from itertools import product

In [44]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        #for v in self.__variables:
        #    print('%s=%i' % (v, self.Value(v)), end=' ')
        #print()

    def solution_count(self):
        return self.__solution_count

In [171]:
def create_variables(model, dimensions, N):
    x = {}
    for var_id in product(*dimensions.values()):
        x[var_id] = model.NewIntVar(0, N, f'{var_id}')

    return x

In [173]:
def add_constraint(model, ageb, dimensions, variables, c_name, c_dict, verbose=False):
    assert c_name in ageb.keys()
    
    for k, v in c_dict.items():
        assert k in dimensions.keys()
        for vv in v:
            assert vv in dimensions[k]
    
    # Replace classes with classes involved in the constraint only
    c_dims = dimensions.copy()
    for d in c_dims.keys():
        if d in c_dict.keys():
            c_dims[d] = c_dict[d]
    
    # Find all variables involved in the constraints
    c_vars = []
    for var_id in product(*c_dims.values()):
        c_vars.append(variables[var_id])
    
    if verbose:
        print(' + '.join([v.Name() for v in c_vars]) + f' = {c_name}')
    
    model.Add(sum(c_vars) == ageb[c_name])

In [174]:
model = cp_model.CpModel()

variables = create_variables(model, dimensions, ageb['POBTOT'])

for c_name, c_dict in constraints.items():
    add_constraint(model, ageb, dimensions, variables, c_name, c_dict, verbose=False)
    
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(list(x.values()))
solver.parameters.enumerate_all_solutions = True

status = solver.Solve(model, solution_printer)
print(f'Found {solution_printer.solution_count()} solutions. Status: {status}')

Found 613872 solutions. Status: 4
